In [1]:


# # # Foreacasting with LSTM
#
# In depth evaluation and testing of models supplemented by graphs, plots and tables of MAPE for different time steps ahead in terms of prediction.
#
# The problem framing:
# forecast periods p = {6, 12, 18, 24, 30, 36} hours
#
# - predict the period:
#     - single step of p
#     - multistep the consecutive p step values

In [2]:




get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [3]:




import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [4]:




from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [5]:




from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import RMSprop, Adam, SGD, Adagrad

Using TensorFlow backend.


In [6]:




from utils import *

In [7]:




plt.rcParams['figure.figsize'] = (20, 10)

In [8]:







# ### Load preprocessed data into dataframe

In [9]:




filename = 'processed_series.csv'
df = pd.read_csv(filename,
                 low_memory=False,
                 index_col='Date', parse_dates=True
                )
df.columns = ['Value']

print(df.shape)
df.head()


# ### Resample to 6 hours

(395905, 1)


,Value
Date,
1975-01-01 00:00:00,7.5945
1975-01-01 01:00:00,7.5925
1975-01-01 02:00:00,7.5905
1975-01-01 03:00:00,7.5885
1975-01-01 04:00:00,7.5870


In [10]:




df_six_hr = df.Value.resample('6H').mean().to_frame()

In [11]:




print('DF shape:', df_six_hr.shape)
df_six_hr.head()

DF shape: (65985, 1)


,Value
Date,
1975-01-01 00:00:00,7.589750
1975-01-01 06:00:00,7.580583
1975-01-01 12:00:00,7.571958
1975-01-01 18:00:00,7.565625
1975-01-02 00:00:00,7.558917


In [12]:




df_six_hr.describe()


# ### Transformations
#
# Scale using StandardScaler

,Value
count,65985.000000
mean,7.464822
std,0.278341
min,6.998333
25%,7.258000
50%,7.418417
75%,7.619250
max,10.065917


In [13]:




scaler = StandardScaler()
scaled = scaler.fit_transform(df_six_hr)

In [14]:




df_scaled = df_six_hr.copy()
df_scaled[:] = scaled
df_scaled.head()

,Value
Date,
1975-01-01 00:00:00,0.448832
1975-01-01 06:00:00,0.415899
1975-01-01 12:00:00,0.384911
1975-01-01 18:00:00,0.362157
1975-01-02 00:00:00,0.338056


In [15]:




df_scaled.describe()

,Value
count,6.598500e+04
mean,1.820264e-15
std,1.000008e+00
min,-1.675973e+00
25%,-7.430587e-01
50%,-1.667236e-01
75%,5.548181e-01
max,9.345052e+00


In [16]:







# ### Supervised learning framing
#
# Extract supervised learning data

In [17]:




n_inputs = 8 # Use the last e.g 48 hours data
n_outputs = 1 # Predict the next hours
forecast_column = 'Value'

df_data, label_columns = frame_supervised_data(df_scaled, n_inputs, n_outputs,
                                                    forecast_columns=[forecast_column])
print(df_data.columns)
df_data.head()

Index(['Value(t-8)', 'Value(t-7)', 'Value(t-6)', 'Value(t-5)', 'Value(t-4)',
       'Value(t-3)', 'Value(t-2)', 'Value(t-1)', 'Value(t)'],
      dtype='object')


,Value(t-8),Value(t-7),Value(t-6),Value(t-5),Value(t-4),Value(t-3),Value(t-2),Value(t-1),Value(t)
Date,,,,,,,,,
1975-01-03 00:00:00,0.448832,0.415899,0.384911,0.362157,0.338056,0.292398,0.248088,0.208269,0.169197
1975-01-03 06:00:00,0.415899,0.384911,0.362157,0.338056,0.292398,0.248088,0.208269,0.169197,0.142701
1975-01-03 12:00:00,0.384911,0.362157,0.338056,0.292398,0.248088,0.208269,0.169197,0.142701,0.111714
1975-01-03 18:00:00,0.362157,0.338056,0.292398,0.248088,0.208269,0.169197,0.142701,0.111714,0.091205
1975-01-04 00:00:00,0.338056,0.292398,0.248088,0.208269,0.169197,0.142701,0.111714,0.091205,0.075487


In [18]:




# Extract features
X = df_data.drop(label_columns, axis=1)

# Extract labels
y = df_data[label_columns]

X.shape, y.shape


# #### Split into train and test sets
#
# Using the first 40 years for training, and remaining 6 years for testing.

((65977, 8), (65977, 1))

In [19]:




split_year = '2014'

X_train = np.expand_dims(X[:split_year].values, axis=2)
X_test = np.expand_dims(X[split_year:].values, axis=2)

y_train = y[:split_year].values[:, 0]
y_test = y[split_year:].values[:, 0]

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (58432, 8, 1)
y_train shape (58432,)
X_test shape (9005, 8, 1)
y_test shape (9005,)


In [20]:




n_steps = 6
dataset = (X_train, y_train, X_test, y_test)

In [21]:








# ### Define the network model

In [22]:




model = Sequential()

model = Sequential()
model.add( LSTM(128, activation='relu', input_shape=X_train.shape[1:]) )
model.add(Dropout(0.2))
model.add( Dense(1) ) # Linear activation, for regression

optimizer = RMSprop(lr=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [23]:




print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:




n_epochs = 15
# set shuffle to False!
h = model.fit(X_train, y_train, epochs=n_epochs,
              batch_size=20,
              validation_split=0.3, shuffle=False)

Train on 40902 samples, validate on 17530 samples
Epoch 1/15
40902/40902 [==============================] - 13s 320us/step - loss: 0.2089 - val_loss: 0.1042
Epoch 2/15
40902/40902 [==============================] - 13s 313us/step - loss: 0.0831 - val_loss: 0.0676
Epoch 3/15
40902/40902 [==============================] - 13s 322us/step - loss: 0.0572 - val_loss: 0.0455
Epoch 4/15
40902/40902 [==============================] - 12s 305us/step - loss: 0.0430 - val_loss: 0.0378
Epoch 5/15
40902/40902 [==============================] - 12s 292us/step - loss: 0.0342 - val_loss: 0.0317
Epoch 6/15
40902/40902 [==============================] - 12s 291us/step - loss: 0.0295 - val_loss: 0.0262
Epoch 7/15
40902/40902 [==============================] - 12s 302us/step - loss: 0.0263 - val_loss: 0.0255
Epoch 8/15
40902/40902 [==============================] - 12s 302us/step - loss: 0.0247 - val_loss: 0.0259
Epoch 9/15
40902/40902 [==============================] - 13s 326us/step - loss: 0.0239 - val_

In [ ]:




plt.plot(h.history['loss'], label='Train')
plt.plot(h.history['val_loss'], label='Validation')

plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

In [ ]:




def recursive_forecast(model, X_test, y_test, n_steps=1):
    # Make an accumulator for predictions
    predictions = np.zeros(shape=(y_test.shape[0], n_steps, 1))
    predictions[:] = np.nan

    X_test_step = X_test.copy()

    for i in range(n_steps):
        predictions[:, i] = model.predict(X_test_step)
        X_test_step = np.concatenate((X_test_step[:, 1:], predictions[:, i:i+1]), axis=1)

    return predictions

In [ ]:




# Make recursive multi-step predictions
y_pred_multi = recursive_forecast(model, X_test, y_test, n_steps)

# Evaluate
lstm_rmse = eval_multi(y_test, y_pred_multi, calc_rmse, scaler)
lstm_mape = eval_multi(y_test, y_pred_multi, calc_mape, scaler)

# Report the metrics
metrics = np.array([lstm_rmse, lstm_mape]).T
summary = report_metrics(metrics, ['RMSE', 'MAPE'])

In [ ]:




summary

In [ ]:


summary.plot()

In [ ]:




visualize_pred(y_test, y_pred_multi, 'LSTM Model', y, split_year, scaler)

In [ ]:


visualize_pred_ext(y_test, y_pred_multi, 'LSTM Model', y, split_year, scaler)

In [ ]:







# ### Results summary
# MAPE values

In [ ]:




results = [summary]
names = ['LSTM']

mape_results = [res.MAPE for res in results]

In [ ]:




summary = pd.concat(mape_results, axis=1)
summary.columns = names

In [ ]:




summary.T